In [8]:
import sys
import warnings
import pandas as pd
import numpy as np
import synapseclient as sc
from utils import save_data_to_synapse, fix_column_name
from synapseclient import Entity, Project, Folder, File, Link, Activity
import time
import os
from datetime import datetime
warnings.simplefilter("ignore")

syn = sc.login()

Welcome, aryton tediarjo!



INFO:synapseclient_default:Welcome, aryton tediarjo!



In [13]:
EMS_DEMOGRAPHIC_TABLE = "syn10295288"
RAW_FEATURIZED_EMS_TABLE = "syn21256442"


In [42]:
demographic_data   = syn.tableQuery("SELECT distinct(healthCode) as \
                                    healthCode, dataGroups as PD, LOWER('gender.json.answer') as gender from {} \
                                    where dataGroups NOT LIKE '%test_user%'".format(EMS_DEMOGRAPHIC_TABLE))

In [43]:
demographic_data   = demographic_data.asDataFrame().drop_duplicates("healthCode", keep = "first").reset_index(drop = True)
demographic_data["PD"] = demographic_data["PD"].map({"ms_patient":1, "control":0})
demographic_data["age"] = np.NaN
demographic_data = demographic_data[["healthCode", "PD", "gender", "age"]]

In [44]:
demographic_data

,healthCode,PD,gender,age
0,bb6546a7-0091-4c8b-9285-b60d4e360732,0,male,NaN
1,76acb4c3-6254-4264-a963-93c52858e8ce,1,male,NaN
2,4c6aa94d-0ddb-414c-ac7f-135106f15721,1,female,NaN
3,22cf36f5-6ca6-463b-bc31-4887b5e22481,1,male,NaN
4,3587578a-3dc2-4177-a627-1b9fa5611f6e,0,female,NaN
5,f77b6687-de08-4c3d-8709-d57c09b6f8f7,1,female,NaN
6,2ac1a84e-20cd-4736-93ba-f08796678752,1,female,NaN
7,d02debb7-9429-4d2c-bc92-846f10d4e6ce,1,female,NaN
8,4e9ceb21-75d8-46c9-844b-da70af31c8a3,1,male,NaN
9,b5ca4dc5-fec4-4752-a9e4-80ea2f8f4255,1,male,NaN
